Nombre:Marcela Cabrera

# Ejercicio 9: Uso de la API de Google Gemini

En este ejercicio vamos a aprender a utilizar la API de OpenAI

## 1. Uso básico

El siguiente código sirve para conectarse con la API de Google Gemini de forma básica

In [ ]:
import google.generativeai as genai

# Leer la API key desde un archivo de texto
with open('api-key.txt', 'r') as file:
    api_key = file.read().strip()

# Configurar la API de Google Gemini
genai.configure(api_key=api_key)

# Crear el modelo (usando el más reciente y estable)
model = genai.GenerativeModel('gemini-2.5-flash')

## 2. Retrieval

### 2.1 Cargo el corpus de 20 News Groups

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# Cargar el corpus de noticias sin cabeceras, pies de página y citas
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = newsgroups.data
df = pd.DataFrame(docs, columns=['doc'])
df.head(10)


### 2.2 Transformo a embeddings



1.  Normalizar el corpus




In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["doc"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["doc_norm"] = df["doc"].astype(str).map(normalize_text)

df.head()


2. Definir una función chunk_text, y dividir los textos en chunks.



In [ ]:
def chunk_doc(doc: str, max_chars: int = 800, overlap: int = 100):

    chunks = []
    start = 0
    n = len(doc)
    while start < n:
        end = min(start + max_chars, n)
        chunk = doc[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_doc(row["doc_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

In [ ]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

### 2.3 Creo una query y hago la búsqueda

In [ ]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "¿De qué se tratan los documentos recuperados?"

query_vec = embed_query(query_text)
query_vec.shape

In [ ]:
print(embeddings.shape, embeddings.dtype)

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(query_vec, k=10)

Obtengo los 5 documentos más similares a mi query

In [ ]:
# Recuperamos los fragmentos más relevantes según la consulta
top_k = 5
D, I = index.search(query_vec, k=top_k)

top_idxs = I.flatten().tolist()
top_passages = chunks_df.iloc[top_idxs].copy()
top_passages["distance"] = D.flatten()
top_passages[["doc_id", "chunk_id", "distance", "text"]].head(10)

In [ ]:
passages_text = []
for i, row in enumerate(top_passages.itertuples(index=False), 1):
    passages_text.append(f"Fragmento {i}:\n{row.text}")

context_block = "\n\n".join(passages_text)

In [ ]:

prompt = f"""
Eres un asistente que analiza textos.

PREGUNTA DEL USUARIO:
{query_text}

PASAJES (top {top_k}):
{context_block}

"""

# Llamada a Gemini (usando el modelo ya creado)
response = model.generate_content(prompt)

print(response.text)
